In [1]:
import os
import shutil
import numpy as np
import imageio.v2 as imageio
import skimage
import matplotlib.pyplot as plt
from glob import glob
from scipy import io
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
from tensorflow import keras
import keras.backend as K
from sklearn.preprocessing import normalize
from sklearn import metrics

In [2]:
gpu_no = '/gpu:0'
# gpu_no = '/cpu:0'

n_filter = 16 # 필터의 개수(특징 맵의 개수)
kernel1 = (3, 3) # 커널 사이즈 (주로 2개의 정수가 들어있는 튜플, 혹은 리스트 형태)
padding1 = 'same' #"valid" 혹은 "same"
strides1 = (1, 1) # 움직이는 거리 (= convolution layer의 size)
input_shape = (128, 128, 3) # 계층을 처음 사용할 때, 4-D 데이터에서 batches값을 뺀 (rows,cols,channels)를 받음 이미지는 주로 채널 3개(RGB)
pool_size1 = (2, 2) # pooling size = 2이면 크기가 절반으로 줄어드는 출력층을 만들어 냄.

learning_rate = 0.00001
batch_size = 4 # batch size란 하나의 소그룹게 속하는 데이터 수(작은 단위로 나눠서 학습 시간을 단축)
n_epochs = 20 # epoch는 전체 트레이닝 셋이 신경망을 통과할 횟수 

In [3]:
def preprocessing(png_list, num):
    if (num == 0) or (num == 1) :
        globals()['img_list' + str(num)] = []
        globals()['label_list' + str(num)] = [] # 0 = LA, 1 = SA
        for jj, ffname in enumerate(png_list):
            img1 = imageio.imread(ffname)
            img2 = skimage.transform.resize(img1, (128, 128), anti_aliasing=True, preserve_range=False)    
            # img2 = img2/np.max(img2) # normalize image to [0 1]
            # print(img1.shape, img2.shape)

            globals()['img_list' + str(num)].append(img2)

            globals()['label_list' + str(num)].append(num)
    else:
        return # exit the func

In [4]:
import random

axis_views = ['long', 'short']
compare_image_list = []
for i, j in zip(axis_views, range(len(axis_views))) : # n값 변환에 따라 데이터 수가 많아질 때, 해당 코드의 확장 가능성 있음.
    initial = (i[0]).upper() # j = (i.capitalize())[0]
    png_list = 'png_'+ initial +'A_list'
    globals()[png_list] = glob('data/' + i + 'axis_' + 'n=200' + '/*.png')
    preprocessing(globals()[png_list], j)  # 0 = LA, 1 = SA
    

In [5]:
def grayscale_to_rgb(images, channel_axis=-1):
    images = K.expand_dims(images, axis=channel_axis)
    tiling = [1]*4 # dimensions: B, H, W, C
    tiling[channel_axis] *= 3
    images = K.tile(images, tiling)
    return images

In [6]:
img_list = img_list0 + img_list1
label_list = label_list0 + label_list1
png_fname_list = png_LA_list + png_SA_list
X = np.stack(img_list, axis=0)
y = np.stack(label_list, axis=0)
z = np.stack(png_fname_list, axis=0)
# X = grayscale_to_rgb(X)

In [7]:
def model_fn(model_num):
        model = Sequential()
        model.add( Conv2D( filters = n_filter, kernel_size = kernel1, padding = padding1, strides = strides1, input_shape = input_shape) )

        if model_num == 1:
            model.add( BatchNormalization())

        model.add( Activation('relu'))
        model.add( MaxPooling2D(pool_size = pool_size1) )
        model.add( Conv2D(filters=2*n_filter, kernel_size=kernel1, padding=padding1, strides=strides1))

        if model_num == 1:
            model.add( BatchNormalization())

        model.add( Activation('relu')) # 0 이하의 값은 다음 레이어에 전달하지 않습니다. 0이상의 값은 그대로 출력합니다.
        model.add( MaxPooling2D(pool_size = pool_size1))
        model.add( Flatten())
        model.add( Dense(64))
        model.add( Activation('relu'))
        model.add( Dropout(0.5))
        model.add(Dense(2, activation='sigmoid')) # 입력 값을 0과 1 사이의 값으로 변환하여 출력합니다.
        # mertrics_list = [tf.keras.metrics.AUC(), 'accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tfa.metrics.F1Score(num_classes = 2, average='macro')]
        model.compile ( optimizer = Adam(learning_rate = learning_rate), loss = 'binary_crossentropy',  metrics=['accuracy'])
        
        return model
    
    
    

In [8]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

gpu_no = '/gpu:0'
acc_list = []
specificity_list = []
precision_list = []
recall_list = []
f1score_list = []
roc_score_list = []

model_num = 1
kf = KFold(n_splits=5, shuffle = True) #n개의 Fold로 나눔

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_train3 = grayscale_to_rgb(X_train)
    X_test3 = grayscale_to_rgb(X_test)
    model = model_fn(1)
    
    y_train_encode = to_categorical(y_train)
    y_test_encode = to_categorical(y_test)
    
    with K.tf.device(gpu_no):
        history = model.fit(x = X_train3, y = y_train_encode,
                    epochs = n_epochs,
                    batch_size = batch_size,
                    validation_data=(X_test3, y_test_encode))
    
        acc = model.evaluate(X_test3, y_test_encode)[1]
        
        prob_score = model.predict(X_test3, batch_size=1, verbose=0)
        y_pred = np.zeros((prob_score.shape[0], 1))
        y_pred[prob_score[:,1]>=0.5] = 1
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        
        specificity=tn / (tn+fp)
        precision = tp / (tp + fp)
        recall = tp / (tp+fn)
        f1score = (2*precision*recall)/(precision+recall)
        roc_score = roc_auc_score(y_test, y_pred)
    
    acc_list.append(acc)
    specificity_list.append(specificity)
    precision_list.append(precision)
    recall_list.append(recall)
    f1score_list.append(f1score)
    roc_score_list.append(roc_score)
    
    
    model_num += 1

2022-12-07 22:45:32.071581: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-07 22:45:32.089865: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-07 22:45:32.090228: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-07 22:45:32.091367: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the app

Epoch 1/20


2022-12-07 22:45:34.082272: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-12-07 22:45:36.813433: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


80/80 [==============================] - 5s 7ms/step - loss: 0.6574 - accuracy: 0.6719 - val_loss: 0.6482 - val_accuracy: 0.8625
Epoch 2/20
80/80 [==============================] - 0s 5ms/step - loss: 0.3659 - accuracy: 0.8750 - val_loss: 0.5656 - val_accuracy: 0.7875
Epoch 3/20
80/80 [==============================] - 0s 5ms/step - loss: 0.2651 - accuracy: 0.9000 - val_loss: 0.4845 - val_accuracy: 0.8125
Epoch 4/20
80/80 [==============================] - 0s 6ms/step - loss: 0.2050 - accuracy: 0.9406 - val_loss: 0.3847 - val_accuracy: 0.8500
Epoch 5/20
80/80 [==============================] - 0s 6ms/step - loss: 0.1533 - accuracy: 0.9531 - val_loss: 0.2729 - val_accuracy: 0.8875
Epoch 6/20
80/80 [==============================] - 0s 5ms/step - loss: 0.1446 - accuracy: 0.9594 - val_loss: 0.2060 - val_accuracy: 0.9125
Epoch 7/20
80/80 [==============================] - 0s 5ms/step - loss: 0.0906 - accuracy: 0.9812 - val_loss: 0.1606 - val_accuracy: 0.9250
Epoch 8/20
80/80 [=============

In [9]:
data_list = [acc_list, specificity_list, precision_list, recall_list, f1score_list, roc_score_list]
for i in data_list:
    print(i)

[0.949999988079071, 0.987500011920929, 0.9375, 0.9750000238418579, 0.987500011920929]
[1.0, 1.0, 0.9347826086956522, 0.9428571428571428, 1.0]
[1.0, 1.0, 0.918918918918919, 0.9574468085106383, 1.0]
[0.8974358974358975, 1.0, 1.0, 1.0, 0.9736842105263158]
[0.945945945945946, 1.0, 0.9577464788732395, 0.9782608695652174, 0.9866666666666666]
[0.9487179487179487, 1.0, 0.9673913043478262, 0.9714285714285714, 0.986842105263158]


In [10]:
    import math

    # 평균 구하기 함수
    def mean(values):
        if len(values) == 0:
            return None
        return sum(values, 0.0) / len(values)


    # 표준 편차 구하기 함수
    def StandardDeviation(values, option):
        if len(values) < 2:
            return None
        sd = 0.0
        sum = 0.0
        meanValue = mean(values)

        for i in range(0, len(values)):
            diff = values[i] - meanValue
        sum += (diff * diff)

        sd = math.sqrt(sum / (len(values) - option))
        return sd

In [11]:
import pandas as pd
data_list = [acc_list, specificity_list, precision_list, recall_list, f1score_list, roc_score_list]

col = ["Mean","S.D"]
ind = ["accuracy","specificity","AUC","precision","recall","f1 score"]
con = []
for i in data_list:
    con.append([mean(i),StandardDeviation(i, 1)])
df = pd.DataFrame(con,columns=col,index=ind)
print(df)

                 Mean       S.D
accuracy     0.967500  0.010000
specificity  0.975528  0.012236
AUC          0.975273  0.012363
precision    0.974224  0.000270
recall       0.973724  0.006471
f1 score     0.974876  0.005983


In [12]:
df.transpose()

,accuracy,specificity,AUC,precision,recall,f1 score
Mean,0.9675,0.975528,0.975273,0.974224,0.973724,0.974876
S.D,0.0100,0.012236,0.012363,0.000270,0.006471,0.005983


In [13]:
df

,Mean,S.D
accuracy,0.967500,0.010000
specificity,0.975528,0.012236
AUC,0.975273,0.012363
precision,0.974224,0.000270
recall,0.973724,0.006471
f1 score,0.974876,0.005983
